#### Attribution

All of the slides in this module are adaptations on materials found at [https://musicinformationretrieval.com/index.htm](https://musicinformationretrieval.com/index.html) and [https://github.com/stevetjoa/musicinformationretrieval.com](https://github.com/stevetjoa/musicinformationretrieval.com) which have been shared under the [MIT license](https://github.com/stevetjoa/musicinformationretrieval.com/blob/gh-pages/LICENSE).

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np, scipy, matplotlib.pyplot as plt, sklearn, urllib, IPython.display as ipd
import pandas as pd
import librosa, librosa.display
plt.rcParams['figure.figsize'] = (15, 5) # Set default plot size
import seaborn as sns

In [2]:
sns.__version__

'0.11.0'

## Steps in Building a Classifier

1. Obtain training data, in this case a number of samples of snare and bass drums
2. Extract features from these samples that capture the sonic differences.
3. Train and validate a model

[&larr; Back to Index](index.html)

# Step 1- Obtain Train Data

Here we will use the 20 recordings of bass and snare drums from [https://musicinformationretrieval.com/](https://musicinformationretrieval.com/)


# Step 1- Basic Feature Extraction

Somehow, we must extract the characteristics of our audio signal that are most relevant to the problem we are trying to solve. For example, if we want to classify instruments by timbre, we will want features that distinguish sounds by their timbre and not their pitch. If we want to perform pitch detection, we want features that distinguish pitch and not timbre.

This process is known as feature extraction.

Let's begin with twenty audio files: ten kick drum samples, and ten snare drum samples. Each audio file contains one drum hit.

Read and store each signal:

In [2]:
# Note: Ignore the "PySoundFile failed. Trying audioread instead." error
#       which always happens when loading mp3 files
kick_signals = [
    librosa.load(p)[0] for p in Path().glob('audio/drum_samples/train/kick_*.mp3')
]
snare_signals = [
    librosa.load(p)[0] for p in Path().glob('audio/drum_samples/train/snare_*.mp3')
]

/Users/dm6258xw/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


AttributeError: 'PosixPath' object has no attribute 'encode'

In [ ]:
len(kick_signals)

In [ ]:
len(snare_signals)

Display the kick drum signals:

In [ ]:
plt.figure(figsize=(15, 6))
for i, x in enumerate(kick_signals):
    plt.subplot(2, 5, i+1)
    librosa.display.waveplot(x[:10000])
    plt.ylim(-1, 1)

Display the snare drum signals:

In [ ]:
plt.figure(figsize=(15, 6))
for i, x in enumerate(snare_signals):
    plt.subplot(2, 5, i+1)
    librosa.display.waveplot(x[:10000])
    plt.ylim(-1, 1)

## Constructing a Feature Vector

A *feature vector* is simply a collection of features. Here is a simple function that constructs a two-dimensional feature vector from a signal:

In [ ]:
def extract_features(signal):
    return (
        librosa.feature.zero_crossing_rate(signal)[0, 0],
        librosa.feature.spectral_centroid(signal)[0, 0],
    )

If we want to aggregate all of the feature vectors among signals in a collection, we can use a list comprehension as follows:

In [ ]:
kick_zcr, kick_spec_cent = zip(*np.array([extract_features(x) for x in kick_signals]))
snare_zcr, snare_spec_cent = zip(*np.array([extract_features(x) for x in snare_signals]))

In [ ]:
drum_samples = pd.DataFrame({'type': 10*['kick'] + 10*['snare']
                           ,'zcr' : np.concatenate((kick_zcr, snare_zcr))
                           ,'spec_cent' : np.concatenate((kick_spec_cent, snare_spec_cent))
                           })

Visualize the differences in features by plotting separate histograms for each of the classes:

In [ ]:
assert sns.__version__ == '0.11.0', "Run: conda install -c anaconda seaborn"

In [ ]:
sns.pairplot(drum_samples, hue="type")

## Feature Scaling

The features that we used in the previous example included zero crossing rate and spectral centroid. These two features are expressed using different units. 

In [ ]:
drum_samples[["zcr", "spec_cent"]].describe()

This discrepancy can pose problems when performing classification later. Therefore, we will normalize each feature vector to a common range and store the normalization parameters for later use.  

Many techniques exist for scaling your features. For now, we'll use [`sklearn.preprocessing.MinMaxScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).  `MinMaxScaler` returns an array of scaled values such that each feature dimension is in the range -1 to 1.

Scale each feature dimension to be in the range -1 to 1:

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
drum_samples[['zcr', 'spec_cent']] = scaler.fit_transform(drum_samples[['zcr', 'spec_cent']])

In [ ]:
drum_samples[["zcr", "spec_cent"]].describe()

Plot the scaled features:

In [ ]:
sns.pairplot(drum_samples, hue="type")

## Fitting and Testing a Model

#### Create a validation set

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 

data_train, data_test, labels_train, labels_test = train_test_split(drum_samples[["zcr", "spec_cent"]],drum_samples.type, test_size=0.3)

#### Fit a logistic regression model

In [ ]:
# Initialize Classifier. 
LRC = LogisticRegression()
LRC.fit(data_train, labels_train)

#### Use the model to classify the test data

In [ ]:
# Test classifier with the test data
predicted = LRC.predict(data_test)

#### Evaluate the model's performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(labels_test, predicted))

In [ ]:

confusion_matrix(predicted, labels_test)

[&larr; Back to Index](index.html)